In [1]:
# РЕПОЗИТОРИЙ НА GITHUB https://github.com/ArturWD/HSE-Python/



import pandas as pd
from datetime import datetime

# Импорт доп данных
extraData = pd.read_csv("./Data/CASC_Constant.csv", sep=",")
# Конвертация строки в дату
extraData['ActivationDate'] = pd.to_datetime(extraData['ActivationDate'])

# Импорт полных данных
mainData = pd.read_csv("./Data/casc-resto.csv", sep=";" , decimal=",")
# Импорт сокращённых данных для тестирования
#mainData = pd.read_csv("lab-1/casc-resto-small.csv", sep=",")

# Конвертирование строки в дату
mainData['RKDate'] = pd.to_datetime(mainData['RKDate'])
#mainData['SummAfterPointsUsage'] = mainData['SummAfterPointsUsage'].astype(float)
#mainData['SummBasic'] = mainData['SummBasic'].astype(float)

print(mainData.info())

# Создать фиксированную дату
fixed_date = datetime.strptime('2017-07-01', '%Y-%m-%d')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 882222 entries, 0 to 882221
Data columns (total 9 columns):
CustomerID              882222 non-null int64
Restaurant              882222 non-null int64
RKDate                  882222 non-null datetime64[ns]
RegionName              882222 non-null object
BrandsNames             882222 non-null object
DishCategoryName        881608 non-null object
Quantity                882222 non-null int64
SummBasic               882222 non-null float64
SummAfterPointsUsage    882222 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(3), object(3)
memory usage: 60.6+ MB
None


In [2]:
from dateutil import relativedelta

# Покупал или нет за период времени
def isInDateRange(dates):
    for date in dates:
        if date <=  datetime.strptime('2017-12-31', '%Y-%m-%d')  and date >= fixed_date:
            return 1
    return 0

# Как давно покупал в последний раз
def getRecency(dates):
    minDiff = 10000 
    for date in dates:
        if date <= fixed_date and (fixed_date - date).days < minDiff:
            
            minDiff = (fixed_date - date).days
    return minDiff
# Как часто покупал в месяц
def getFrequency(dates):
    datesBefore = list(filter(lambda d: d <= fixed_date, dates))
    if len(datesBefore) == 0:
        return 0
    
    months  = 1 + (relativedelta.relativedelta(fixed_date, min(datesBefore)) ).months
       
    return len(datesBefore) / months

# 
def getMonetaryValue(group):
    spent = 0
    visitedTimes = 1
    
    for index, row in group.iterrows():
        if (row['RKDate'] <= fixed_date):
            visitedTimes+=1
            spent+=row['SummAfterPointsUsage']
        
    return spent / visitedTimes

# Новая переменная. Показывает, сколько в среднем было оплачено бонусами
def getAvgNonCashPay(group):
    nonCashSpent = 0
    visitedTimes = 1
    
    for index, row in group.iterrows():
        if (row['RKDate'] <= fixed_date):
            visitedTimes+=1
            nonCashSpent+= ( row['SummBasic'] - row['SummAfterPointsUsage'] )
        
    return nonCashSpent / visitedTimes


# Агрегация данных
def f(group):
    d = {}
    d['did_buy'] = isInDateRange(group['RKDate'])
    d['Recency'] = getRecency(group['RKDate'])
    d['Frequency'] = getFrequency(group['RKDate'])
    d['MonetaryValue'] = getMonetaryValue(group)
    d['AvgNonCashPay'] = getAvgNonCashPay(group)
    return pd.Series(d, index=['did_buy', 'Recency', 'Frequency', 'MonetaryValue', 'AvgNonCashPay'])

groupedData = mainData.groupby("CustomerID").apply(f)
aggregatedData= groupedData.reset_index()


print(aggregatedData)

      CustomerID  did_buy  Recency  Frequency  MonetaryValue  AvgNonCashPay
0        2728046      0.0    160.0   8.142857     247.448276       2.965517
1        2728088      0.0    720.0   4.857143     271.657143       0.000000
2        2728089      1.0     15.0   4.714286     387.007353      22.757353
3        2728095      0.0    177.0  14.142857     260.110000      26.670000
4        2728107      1.0    115.0   3.000000     375.818182      19.681818
...          ...      ...      ...        ...            ...            ...
9995     2913103      0.0    401.0   9.750000     432.450000       6.600000
9996     2913106      0.0    126.0  46.750000     251.638298      29.569149
9997     2913114      0.0     95.0  27.750000     309.089286      33.696429
9998     2913123      1.0      8.0  58.000000     128.678112      11.892704
9999     2913132      1.0     44.0   2.571429     350.736842       0.000000

[10000 rows x 6 columns]


In [3]:
import datetime as dt
import numpy as np

# слияние агрегированных данных с дополнительными данными
dataSet = pd.merge(aggregatedData, extraData, left_on='CustomerID', right_on='CustomerId', how='inner') 

# удаление колонок с идентификаторами, т.к. они не нужны
del dataSet['CustomerId']
del dataSet['CustomerID']

# замена строковых значений на бинарные
sex = pd.get_dummies(dataSet['Sex'],drop_first=True)
dataSet.drop(['Sex'],axis=1,inplace=True)
dataSet = pd.concat([dataSet,sex],axis=1)
# конвертация даты
dataSet['ActivationDate'] = pd.to_datetime(dataSet['ActivationDate'])
dataSet['ActivationDate']=dataSet['ActivationDate'].map(dt.datetime.toordinal)

# восполнение пропущенных данных
dataSet['Age'] = dataSet['Age'].fillna(dataSet['Age'].mean())
cols = ['did_buy', 'Age']
dataSet[cols] = dataSet[cols].applymap(np.int64)


print(dataSet.head())


   did_buy  Recency  Frequency  MonetaryValue  AvgNonCashPay  ActivationDate  \
0        0    160.0   8.142857     247.448276       2.965517          735599   
1        0    720.0   4.857143     271.657143       0.000000          735599   
2        1     15.0   4.714286     387.007353      22.757353          735599   
3        0    177.0  14.142857     260.110000      26.670000          735599   
4        1    115.0   3.000000     375.818182      19.681818          735599   

   Age  SubscribedEmail  SubscribedPush  Male  
0   24            False            True     0  
1   46             True            True     1  
2   27             True            True     0  
3   54             True            True     0  
4   48            False           False     1  


In [4]:
from sklearn import datasets
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np

# разбиение выборки на тестовую и обучающую
x_train, x_test, y_train, y_test = train_test_split(dataSet.drop('did_buy',axis=1), dataSet['did_buy'], test_size=0.2, random_state=2)

# создание экземпляра класса логистической регресии и обучение
logisticRegr = LogisticRegression()
logisticRegr.fit(x_train, y_train)


C:\Users\acer\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [6]:
# предсказание результатов по тестовой выборке
print(logisticRegr.predict(x_test))

# оценка качества предсказания
score = logisticRegr.score(x_test, y_test)
print(score)

[0 0 0 ... 0 1 0]
0.655


In [7]:
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier

# оценка показателей precision и recall
knn = KNeighborsClassifier(n_neighbors=8)
knn.fit(x_train, y_train)

y_pred = knn.predict(x_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.82      0.77      1076
           1       0.75      0.64      0.69       924

    accuracy                           0.74      2000
   macro avg       0.74      0.73      0.73      2000
weighted avg       0.74      0.74      0.74      2000

